### TFIDF based Recommendation system

- articles데이터의의 terms of bag of words 파악
- 유저의 읽은 아티클과 관련 단어 추출
- TF-IDF 행렬 생성(읽은 아티클,읽지 않은 아티클) 
- Cosine 유사도 생성(읽은 아티클,읽지 않은 아티클) 
- 추천될 아티클

Describing parameters:

- NEWS_ARTICLES: 데이터 지정
- ARTICLES_READ: 사용자가 읽은 Article_Ids 목록
- NO_RECOMMENDED_ARTICLES: 몇 개의 아티클 추천?


In [ ]:
NEWS_ARTICLES="NewsArticles.csv"
ARTICLES_READ=[1,10] 
NUM_RECOMMENDED_ARTICLES=5

In [ ]:
try:
    import numpy
    import pandas as pd
    import pickle as pk
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import re
    from nltk.stem.snowball import SnowballStemmer
    import nltk
    stemmer = SnowballStemmer("english")
except ImportError:
    print('You are missing some packages! ' \
          'We will try installing them before continuing!')
    !pip install "numpy" "pandas" "sklearn" "nltk"
    import numpy
    import pandas as pd
    import pickle as pk
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import re
    from nltk.stem.snowball import SnowballStemmer
    import nltk
    stemmer = SnowballStemmer("english")
    print('Done!')

#### Text sanitization

- 불용어 제거
- tokenizing
- stemming


In [ ]:
news_articles = pd.read_csv(NEWS_ARTICLES,encoding='unicode_escape',index_col=0)
news_articles.drop(news_articles.columns[news_articles.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
news_articles.head()

,publish_date,article_source_link,title,subtitle,text
article_id,,,,,
1,2017/2/7,http://abcnews.go.com/Politics/pence-break-tie...,"Betsy DeVos Confirmed as Education Secretary, ...",NaN,Michigan billionaire education activist Betsy ...
2,2017/2/7,http://abcnews.go.com/Politics/wireStory/melan...,Melania Trump Says White House Could Mean Mill...,NaN,First lady Melania Trump has said little about...
3,2017/2/7,http://abcnews.go.com/Politics/wireStory/trump...,"As Trump Fears Fraud, GOP Eliminates Election ...",NaN,A House committee voted on Tuesday to eliminat...
4,2017/2/7,http://abcnews.go.com/Politics/appeals-court-d...,Appeals Court to Decide on Challenge to Trump'...,NaN,"This afternoon, three federal judges from the ..."
5,2017/2/7,http://abcnews.go.com/US/23-states-winter-weat...,At Least 4 Tornadoes Reported in Southeast Lou...,NaN,At least four tornadoes touched down in Louisi...


In [ ]:
news_articles.shape

(3824, 5)

In [8]:
news_articles = news_articles[['title','text']].dropna()
articles = news_articles['text'].tolist()
articles[0] #정제 되기 전|

'Michigan billionaire education activist Betsy DeVos was confirmed today to serve as the secretary of education in President Trump\'s administration, after Vice President Mike Pence cast a tie-breaking vote in the Senate. The Senate voted on DeVos"?highly contentious nomination this afternoon, and the tally was split evenly, requiring Pence to use his authority as president of the upper chamber of Congress to break the impasse. This was the first time that a vice president has broken a tie to confirm a Cabinet nominee. Pence read the vote count 50-50 and then voted himself, rendering the tally 51-50. The day before the vote, Democrats staged a 24-hour marathon of speeches, with more than 30 lawmakers taking to the floor to urge at least one additional Republican to vote against DeVos and block her confirmation. "It is hard to imagine a worse choice,"?Sen. Elizabeth Warren, D-Mass., said before she read letters from constituents urging her to vote no. DeVos stirred up vehement oppositio

In [19]:
def clean_text(document):
    document = re.sub('[^\w_\s-]', ' ',document)       #remove punctuation marks and other symbols
    document = re.sub('<.+?>', '', document, 0, re.I|re.S)
    #document = re.sub('&nbsp;| |\t|\r|', '', document)
    tokens = nltk.word_tokenize(document)              #Tokenize 
    cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])    #Stemming
    return cleaned_article

In [ ]:
import nltk
nltk.download('all')


In [20]:
cleaned_articles = list(map(clean_text, articles))
cleaned_articles[0]  #정제가 잘 되었나 확인

'michigan billionair educ activist betsi devo was confirm today to serv as the secretari of educ in presid trump s administr after vice presid mike penc cast a tie-break vote in the senat the senat vote on devo high contenti nomin this afternoon and the talli was split even requir penc to use his author as presid of the upper chamber of congress to break the impass this was the first time that a vice presid has broken a tie to confirm a cabinet nomine penc read the vote count 50-50 and then vote himself render the talli 51-50 the day befor the vote democrat stage a 24-hour marathon of speech with more than 30 lawmak take to the floor to urg at least one addit republican to vote against devo and block her confirm it is hard to imagin a wors choic sen elizabeth warren d-mass said befor she read letter from constitu urg her to vote no devo stir up vehement opposit from teacher union and all 48 senat democrat mani cite concern about her support of school voucher which critic believ will we

#### user가 읽은 아티클 

In [21]:
user_articles = ' '.join(cleaned_articles[i] for i in ARTICLES_READ)
user_articles

'first ladi melania trump has said littl about what she intend to do with her promin posit but in new court document her lawyer say that the multi-year term dure which she is one of the most photograph women in the world could mean million of dollar for her person brand while the new document don t specif mention her term as first ladi the unusu statement about her expect profit drew swift condemn from ethic watchdog as inappropri profit from her high-profil posit which is typic center on public servic the statement came monday in a libel lawsuit the first ladi re-fil in a state trial court in manhattan trump has been su the corpor that publish the daili mail s websit over a now-retract report that claim she onc work as an escort in the file monday trump s lawyer argu that the report was not onli fals and libel but also damag her abil to profit off her high profil and affect her busi opportun trump had the uniqu once-in-a-lifetim opportun as an extrem famous and well-known person as we

### TF-IDF 행렬 생성(읽은 아티클,읽지 않은 아티클)

In [22]:
tfidf_matrix = TfidfVectorizer(stop_words='english', min_df=2) 
article_tfidf_matrix = tfidf_matrix.fit_transform(cleaned_articles)
article_tfidf_matrix 

<3732x18771 sparse matrix of type '<class 'numpy.float64'>'
	with 670119 stored elements in Compressed Sparse Row format>

https://chan-lab.tistory.com/27

In [23]:
# tfidf matrix: 읽은 기사
user_article_tfidf_vector = tfidf_matrix.transform([user_articles])
user_article_tfidf_vector

<1x18771 sparse matrix of type '<class 'numpy.float64'>'
	with 355 stored elements in Compressed Sparse Row format>

In [24]:
user_article_tfidf_vector.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

### Cosine 유사도 생성(읽은 아티클,읽지 않은 아티클)

In [25]:
articles_similarity_score=cosine_similarity(article_tfidf_matrix, user_article_tfidf_vector)

In [26]:
recommended_articles_id = articles_similarity_score.flatten().argsort()[::-1]

In [27]:
recommended_articles_id

array([  10, 1093,    1, ..., 1895, 2879,  276])

In [28]:
#읽은 기사는 추천시스템에서 제거 
final_recommended_articles_id = [article_id for article_id in recommended_articles_id 
                                 if article_id not in ARTICLES_READ ][:NUM_RECOMMENDED_ARTICLES]

In [30]:
final_recommended_articles_id#추천됨

[1093, 55, 233, 372, 3098]

### Article Recommendation

In [31]:
#Recommended Articles and their title
print ('Articles Read')
print (news_articles.loc[ARTICLES_READ]['title'])
print ('\n')
print ('Recommendations ')
print (news_articles.loc[final_recommended_articles_id]['title'])

Articles Read
article_id
1     Betsy DeVos Confirmed as Education Secretary, ...
10    Multi-State Manhunt in Southeast Intensifies f...
Name: title, dtype: object


Recommendations 
article_id
1093             Adele sweeps the boards at Grammy Awards
55      How child predator was caught by tiny clue in ...
233     What goes on in a far-right Facebook filter bu...
372     Beijing, Manila agree on $3.7b in shared projects
3098    Iconic New York Columnist Jimmy Breslin Dead A...
Name: title, dtype: object
